# 📊 PariData — Simulation de trafic routier parisien

**Auteur** : Yannis Albert  
**Objectif** : Analyser les données de comptage routier de Paris, modéliser le trafic, et simuler l'impact de fermetures d'axes sur la redistribution du trafic.

---

## Sommaire
1. [Setup & Installation](#1)
2. [Collecte des données](#2)
3. [Nettoyage & Préparation](#3)
4. [Analyse Exploratoire (EDA)](#4)
5. [Feature Engineering & Modélisation](#5)
6. [Simulation de fermeture d'axes](#6)
7. [Conclusions](#7)

---
<a id='1'></a>
## 1. Setup & Installation

In [ ]:
# Installation des dépendances (Colab)
!pip install -q pandas numpy matplotlib seaborn scikit-learn plotly requests folium

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Style
sns.set_theme(style='darkgrid', palette='viridis')
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['font.size'] = 12

print('✅ Librairies chargées')

---
<a id='2'></a>
## 2. Collecte des données

On utilise l'API Open Data Paris pour récupérer les données de comptage routier.  
En cas d'indisponibilité de l'API, on génère des données de démonstration réalistes basées sur les patterns de trafic parisien.

In [ ]:
import requests

# ============================================================
# AXES ROUTIERS PARISIENS (données réalistes)
# ============================================================
AXES = {
    'AX001': {'nom': 'Boulevard Périphérique — Porte de Vincennes', 'lat': 48.847, 'lon': 2.410, 'capacite': 6000},
    'AX002': {'nom': 'Boulevard Périphérique — Porte de la Chapelle', 'lat': 48.898, 'lon': 2.360, 'capacite': 5800},
    'AX003': {'nom': 'Boulevard Périphérique — Porte d\'Orléans', 'lat': 48.824, 'lon': 2.325, 'capacite': 5500},
    'AX004': {'nom': 'Boulevard Périphérique — Porte Maillot', 'lat': 48.878, 'lon': 2.283, 'capacite': 5700},
    'AX005': {'nom': 'Champs-Élysées', 'lat': 48.870, 'lon': 2.307, 'capacite': 3200},
    'AX006': {'nom': 'Rue de Rivoli', 'lat': 48.860, 'lon': 2.347, 'capacite': 2800},
    'AX007': {'nom': 'Boulevard Saint-Germain', 'lat': 48.853, 'lon': 2.338, 'capacite': 2500},
    'AX008': {'nom': 'Boulevard Haussmann', 'lat': 48.874, 'lon': 2.330, 'capacite': 2600},
    'AX009': {'nom': 'Avenue de la République', 'lat': 48.867, 'lon': 2.377, 'capacite': 2200},
    'AX010': {'nom': 'Quai de Bercy', 'lat': 48.838, 'lon': 2.380, 'capacite': 3000},
    'AX011': {'nom': 'Boulevard Voltaire', 'lat': 48.862, 'lon': 2.380, 'capacite': 2100},
    'AX012': {'nom': 'Avenue des Gobelins', 'lat': 48.836, 'lon': 2.352, 'capacite': 1800},
    'AX013': {'nom': 'Boulevard de Sébastopol', 'lat': 48.863, 'lon': 2.349, 'capacite': 2400},
    'AX014': {'nom': 'Rue Lafayette', 'lat': 48.876, 'lon': 2.350, 'capacite': 2000},
    'AX015': {'nom': 'Boulevard Magenta', 'lat': 48.880, 'lon': 2.357, 'capacite': 2300},
}

# Matrice d'adjacence (connexions entre axes)
ADJACENCY = {
    'AX001': ['AX010', 'AX011', 'AX009'],
    'AX002': ['AX015', 'AX014', 'AX013'],
    'AX003': ['AX012', 'AX007'],
    'AX004': ['AX005', 'AX008'],
    'AX005': ['AX004', 'AX006', 'AX008'],
    'AX006': ['AX005', 'AX007', 'AX013'],
    'AX007': ['AX006', 'AX003', 'AX012'],
    'AX008': ['AX005', 'AX004', 'AX014'],
    'AX009': ['AX001', 'AX011', 'AX015'],
    'AX010': ['AX001', 'AX012'],
    'AX011': ['AX001', 'AX009', 'AX013'],
    'AX012': ['AX003', 'AX007', 'AX010'],
    'AX013': ['AX002', 'AX006', 'AX011'],
    'AX014': ['AX002', 'AX008', 'AX015'],
    'AX015': ['AX002', 'AX009', 'AX014'],
}

print(f'📍 {len(AXES)} axes routiers configurés')
print(f'🔗 {sum(len(v) for v in ADJACENCY.values())} connexions d\'adjacence')

In [ ]:
# ============================================================
# GÉNÉRATION DES DONNÉES DE TRAFIC (30 jours, données horaires)
# ============================================================
np.random.seed(42)

dates = pd.date_range('2025-01-01', periods=30*24, freq='h')
records = []

for date in dates:
    hour = date.hour
    dow = date.dayofweek
    is_weekend = dow >= 5

    for ax_id, ax_info in AXES.items():
        if is_weekend:
            base = ax_info['capacite'] * 0.4
            peak = 1.0 + 0.3 * np.sin(np.pi * (hour - 12) / 12)
        else:
            base = ax_info['capacite'] * 0.5
            morning = np.exp(-0.5 * ((hour - 8) / 1.5)**2) * 0.5
            evening = np.exp(-0.5 * ((hour - 18) / 1.5)**2) * 0.5
            night = 0.15 if (hour < 6 or hour > 22) else 0
            peak = 1.0 + morning + evening - night

        debit = int(base * peak * (1 + np.random.normal(0, 0.1)))
        debit = max(50, min(debit, ax_info['capacite']))
        taux = min(100, round(debit / ax_info['capacite'] * 100, 1))

        records.append({
            'id_arc': ax_id,
            'nom_compteur': ax_info['nom'],
            'date_comptage': date,
            'debit_horaire': debit,
            'taux_occupation': taux,
            'latitude': ax_info['lat'],
            'longitude': ax_info['lon'],
            'capacite': ax_info['capacite'],
        })

df = pd.DataFrame(records)
print(f'✅ Dataset généré : {df.shape[0]:,} lignes × {df.shape[1]} colonnes')
print(f'   Période : {df.date_comptage.min()} → {df.date_comptage.max()}')
print(f'   Axes : {df.id_arc.nunique()}')
df.head()

---
<a id='3'></a>
## 3. Nettoyage & Préparation

In [ ]:
# Vérification des valeurs manquantes
print('📋 Valeurs manquantes :')
print(df.isnull().sum())
print(f'\n📋 Types :')
print(df.dtypes)

In [ ]:
# Ajout des features temporelles
df['heure'] = df['date_comptage'].dt.hour
df['jour_semaine'] = df['date_comptage'].dt.dayofweek
df['jour_nom'] = df['date_comptage'].dt.day_name()
df['mois'] = df['date_comptage'].dt.month
df['is_weekend'] = (df['jour_semaine'] >= 5).astype(int)
df['is_heure_pointe'] = df['heure'].apply(lambda h: 1 if (7<=h<=9) or (17<=h<=19) else 0)

def period(h):
    if 6<=h<10: return 'matin_pointe'
    elif 10<=h<16: return 'journee'
    elif 16<=h<20: return 'soir_pointe'
    elif 20<=h<23: return 'soiree'
    else: return 'nuit'

df['periode'] = df['heure'].apply(period)

print(f'✅ Features temporelles ajoutées')
df.head()

In [ ]:
# Statistiques par axe
axis_stats = df.groupby('id_arc').agg(
    nom_compteur=('nom_compteur', 'first'),
    debit_moyen=('debit_horaire', 'mean'),
    debit_max=('debit_horaire', 'max'),
    debit_std=('debit_horaire', 'std'),
    taux_occ_moyen=('taux_occupation', 'mean'),
    capacite=('capacite', 'first'),
    latitude=('latitude', 'first'),
    longitude=('longitude', 'first'),
).reset_index()

axis_stats['ratio_charge'] = (axis_stats['debit_moyen'] / axis_stats['capacite'] * 100).round(1)
axis_stats = axis_stats.round(1)

print('📊 Top 5 axes les plus chargés :')
axis_stats.sort_values('ratio_charge', ascending=False)[['nom_compteur', 'debit_moyen', 'capacite', 'ratio_charge']].head()

---
<a id='4'></a>
## 4. Analyse Exploratoire (EDA)

In [ ]:
# ============================================================
# 4.1 — Profil horaire moyen du trafic
# ============================================================
hourly = df.groupby(['heure', 'is_weekend'])['debit_horaire'].mean().reset_index()
hourly['type_jour'] = hourly['is_weekend'].map({0: 'Semaine', 1: 'Weekend'})

fig = px.line(
    hourly, x='heure', y='debit_horaire', color='type_jour',
    title='📊 Profil horaire moyen du trafic — Semaine vs Weekend',
    labels={'heure': 'Heure', 'debit_horaire': 'Débit moyen (véh/h)', 'type_jour': ''},
    template='plotly_dark',
    color_discrete_map={'Semaine': '#4FC3F7', 'Weekend': '#FF8A65'},
)
fig.add_vrect(x0=7, x1=9, fillcolor='red', opacity=0.1, annotation_text='Pointe matin')
fig.add_vrect(x0=17, x1=19, fillcolor='red', opacity=0.1, annotation_text='Pointe soir')
fig.show()

In [ ]:
# ============================================================
# 4.2 — Heatmap : trafic par heure et jour de la semaine
# ============================================================
pivot = df.pivot_table(values='debit_horaire', index='jour_semaine', columns='heure', aggfunc='mean')
jours = ['Lundi', 'Mardi', 'Mercredi', 'Jeudi', 'Vendredi', 'Samedi', 'Dimanche']

fig, ax = plt.subplots(figsize=(16, 6))
sns.heatmap(pivot, cmap='YlOrRd', annot=False, fmt='.0f',
            xticklabels=range(24), yticklabels=jours, ax=ax,
            cbar_kws={'label': 'Débit moyen (véh/h)'})
ax.set_title('🗓️ Heatmap du trafic — Heure × Jour de la semaine', fontsize=14, fontweight='bold')
ax.set_xlabel('Heure')
ax.set_ylabel('')
plt.tight_layout()
plt.show()

In [ ]:
# ============================================================
# 4.3 — Top axes par débit moyen
# ============================================================
top_axes = axis_stats.sort_values('debit_moyen', ascending=True)

fig = px.bar(
    top_axes, x='debit_moyen', y='nom_compteur', orientation='h',
    color='ratio_charge', color_continuous_scale='RdYlGn_r',
    title='📈 Débit moyen par axe routier (avec taux de charge)',
    labels={'debit_moyen': 'Débit moyen (véh/h)', 'nom_compteur': '', 'ratio_charge': 'Charge (%)'},
    template='plotly_dark',
)
fig.update_layout(height=500)
fig.show()

In [ ]:
# ============================================================
# 4.4 — Distribution du trafic par période
# ============================================================
fig = px.box(
    df, x='periode', y='debit_horaire',
    category_orders={'periode': ['nuit', 'matin_pointe', 'journee', 'soir_pointe', 'soiree']},
    color='periode', template='plotly_dark',
    title='📦 Distribution du débit par période de la journée',
    labels={'periode': 'Période', 'debit_horaire': 'Débit horaire (véh/h)'},
)
fig.show()

In [ ]:
# ============================================================
# 4.5 — Carte des compteurs avec intensité de trafic
# ============================================================
fig = px.scatter_mapbox(
    axis_stats, lat='latitude', lon='longitude',
    size='debit_moyen', color='ratio_charge',
    hover_name='nom_compteur',
    hover_data={'debit_moyen': ':.0f', 'capacite': True, 'ratio_charge': ':.1f'},
    color_continuous_scale='RdYlGn_r',
    size_max=25, zoom=11.5,
    mapbox_style='carto-darkmatter',
    title='🗺️ Carte du trafic parisien — Intensité par axe',
    template='plotly_dark',
)
fig.update_layout(height=600)
fig.show()

In [ ]:
# ============================================================
# 4.6 — Matrice de corrélation entre axes (débits horaires)
# ============================================================
pivot_corr = df.pivot_table(values='debit_horaire', index='date_comptage', columns='id_arc')
corr = pivot_corr.corr()

# Renommer pour lisibilité
name_map = axis_stats.set_index('id_arc')['nom_compteur'].to_dict()
short_names = {k: v.split('—')[-1].strip()[:20] if '—' in v else v[:20] for k, v in name_map.items()}
corr_display = corr.rename(index=short_names, columns=short_names)

fig, ax = plt.subplots(figsize=(14, 10))
sns.heatmap(corr_display, annot=True, fmt='.2f', cmap='coolwarm', center=0,
            vmin=-1, vmax=1, ax=ax, square=True)
ax.set_title('🔗 Corrélation entre axes routiers (débits horaires)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

---
<a id='5'></a>
## 5. Feature Engineering & Modélisation

In [ ]:
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler

# ============================================================
# Feature Engineering
# ============================================================
df_model = df.copy()

# Encoding cyclique
df_model['heure_sin'] = np.sin(2 * np.pi * df_model['heure'] / 24)
df_model['heure_cos'] = np.cos(2 * np.pi * df_model['heure'] / 24)
df_model['jour_sin'] = np.sin(2 * np.pi * df_model['jour_semaine'] / 7)
df_model['jour_cos'] = np.cos(2 * np.pi * df_model['jour_semaine'] / 7)

# One-hot des axes
dummies = pd.get_dummies(df_model['id_arc'], prefix='axe', drop_first=True)
df_model = pd.concat([df_model, dummies], axis=1)

# Features et cible
feature_cols = ['heure', 'jour_semaine', 'mois', 'is_weekend', 'is_heure_pointe',
                'heure_sin', 'heure_cos', 'jour_sin', 'jour_cos', 'capacite'] + dummies.columns.tolist()

X = df_model[feature_cols]
y = df_model['debit_horaire']

print(f'✅ Features : {X.shape[1]} colonnes')
print(f'   Cible : debit_horaire (mean={y.mean():.0f}, std={y.std():.0f})')

In [ ]:
# ============================================================
# Entraînement et comparaison des modèles
# ============================================================
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_s = scaler.fit_transform(X_train)
X_test_s = scaler.transform(X_test)

models = {
    'Régression Linéaire': LinearRegression(),
    'Random Forest': RandomForestRegressor(n_estimators=200, max_depth=12, random_state=42, n_jobs=-1),
    'Gradient Boosting': GradientBoostingRegressor(n_estimators=200, max_depth=6, learning_rate=0.1, random_state=42),
}

results = []
best_model = None
best_r2 = -1

for name, model in models.items():
    model.fit(X_train_s, y_train)
    y_pred = model.predict(X_test_s)

    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    cv = cross_val_score(model, scaler.transform(X), y, cv=5, scoring='r2')

    results.append({'Modèle': name, 'RMSE': round(rmse, 1), 'MAE': round(mae, 1),
                    'R²': round(r2, 4), 'CV R² (mean)': round(cv.mean(), 4), 'CV R² (std)': round(cv.std(), 4)})

    if r2 > best_r2:
        best_r2 = r2
        best_model = model
        best_name = name
        best_pred = y_pred

    print(f'✅ {name:25s} | RMSE: {rmse:>8.1f} | R²: {r2:.4f} | CV R²: {cv.mean():.4f} ± {cv.std():.4f}')

results_df = pd.DataFrame(results)
print(f'\n🏆 Meilleur modèle : {best_name} (R² = {best_r2:.4f})')
results_df

In [ ]:
# ============================================================
# Prédictions vs Réalité
# ============================================================
fig = go.Figure()
sample_idx = np.random.choice(len(y_test), size=min(500, len(y_test)), replace=False)
sample_idx = np.sort(sample_idx)

fig.add_trace(go.Scatter(y=y_test.iloc[sample_idx].values, mode='markers', name='Réel',
                          marker=dict(color='#4FC3F7', size=4, opacity=0.6)))
fig.add_trace(go.Scatter(y=best_pred[sample_idx], mode='markers', name='Prédit',
                          marker=dict(color='#FF8A65', size=4, opacity=0.6)))
fig.update_layout(title=f'🎯 Prédictions vs Réalité — {best_name}',
                  xaxis_title='Échantillons', yaxis_title='Débit horaire (véh/h)',
                  template='plotly_dark', height=400)
fig.show()

In [ ]:
# ============================================================
# Importance des features
# ============================================================
if hasattr(best_model, 'feature_importances_'):
    fi = pd.DataFrame({'feature': feature_cols, 'importance': best_model.feature_importances_})
    fi = fi.sort_values('importance', ascending=False).head(15)
    fi['importance_pct'] = (fi['importance'] / fi['importance'].sum() * 100).round(1)

    fig = px.bar(fi, x='importance_pct', y='feature', orientation='h',
                 title=f'📊 Importance des features — {best_name}',
                 labels={'importance_pct': 'Importance (%)', 'feature': ''},
                 template='plotly_dark', color='importance_pct',
                 color_continuous_scale='Blues')
    fig.update_layout(height=450, yaxis={'categoryorder': 'total ascending'})
    fig.show()

---
<a id='6'></a>
## 6. 🚧 Simulation de fermeture d'axes

**Scénario** : On ferme un axe routier et on observe comment le trafic se redistribue sur les axes adjacents. La simulation utilise une redistribution proportionnelle basée sur la capacité résiduelle des axes voisins.

In [ ]:
# ============================================================
# Fonction de simulation
# ============================================================
def simulate_closure(axis_stats, adjacency, closed_axis, mode='proportional'):
    """
    Simule la fermeture d'un axe et redistribue le trafic.
    """
    stats = axis_stats.copy()
    neighbors = adjacency.get(closed_axis, [])
    closed_row = stats[stats['id_arc'] == closed_axis].iloc[0]
    traffic = closed_row['debit_moyen']

    print(f'\n🚧 FERMETURE : {closed_row["nom_compteur"]}')
    print(f'   Débit à redistribuer : {traffic:.0f} véh/h')
    print(f'   Axes adjacents : {len(neighbors)}')

    results = []
    for _, row in stats.iterrows():
        r = {
            'id_arc': row['id_arc'],
            'nom': row['nom_compteur'],
            'debit_avant': round(row['debit_moyen']),
            'capacite': row['capacite'],
            'debit_apres': round(row['debit_moyen']),
            'delta': 0,
        }
        if row['id_arc'] == closed_axis:
            r['debit_apres'] = 0
            r['delta'] = -r['debit_avant']
        results.append(r)

    res_df = pd.DataFrame(results)

    # Redistribution proportionnelle
    n_mask = res_df['id_arc'].isin(neighbors)
    if n_mask.sum() > 0:
        residual = (res_df.loc[n_mask, 'capacite'] - res_df.loc[n_mask, 'debit_avant']).clip(lower=1)
        total_res = residual.sum()
        for idx in res_df[n_mask].index:
            ratio = max((res_df.loc[idx, 'capacite'] - res_df.loc[idx, 'debit_avant']) / total_res, 0.05)
            added = traffic * ratio
            res_df.loc[idx, 'debit_apres'] = round(res_df.loc[idx, 'debit_avant'] + added)
            res_df.loc[idx, 'delta'] = round(added)

    res_df['taux_avant'] = (res_df['debit_avant'] / res_df['capacite'] * 100).round(1)
    res_df['taux_apres'] = (res_df['debit_apres'] / res_df['capacite'] * 100).round(1)
    res_df['surcharge'] = (res_df['taux_apres'] - res_df['taux_avant']).round(1)

    # Status
    conditions = [
        res_df['id_arc'] == closed_axis,
        res_df['taux_apres'] > 90,
        res_df['taux_apres'] > 70,
        res_df['taux_apres'] <= 70,
    ]
    choices = ['⛔ FERMÉ', '🔴 Congestion', '🟡 Chargé', '🟢 Fluide']
    res_df['status'] = np.select(conditions, choices, default='🟢 Fluide')

    congested = (res_df['taux_apres'] > 90).sum()
    print(f'   ⚠️  Axes en congestion : {congested}')
    print(f'   📈 Surcharge max : +{res_df["surcharge"].max():.1f}%')

    return res_df

In [ ]:
# ============================================================
# SCÉNARIO 1 : Fermeture des Champs-Élysées
# ============================================================
sim1 = simulate_closure(axis_stats, ADJACENCY, 'AX005')
sim1[sim1['delta'] != 0][['nom', 'debit_avant', 'debit_apres', 'delta', 'taux_avant', 'taux_apres', 'surcharge', 'status']]

In [ ]:
# ============================================================
# Visualisation : Avant / Après fermeture (barres comparatives)
# ============================================================
affected = sim1[sim1['delta'] != 0].copy()
affected['nom_short'] = affected['nom'].apply(lambda x: x.split('—')[-1].strip()[:25] if '—' in x else x[:25])

fig = go.Figure()
fig.add_trace(go.Bar(name='Avant', x=affected['nom_short'], y=affected['debit_avant'],
                      marker_color='#4FC3F7'))
fig.add_trace(go.Bar(name='Après', x=affected['nom_short'], y=affected['debit_apres'],
                      marker_color='#FF8A65'))
# Ligne de capacité
fig.add_trace(go.Scatter(x=affected['nom_short'], y=affected['capacite'],
                          mode='markers+lines', name='Capacité max',
                          line=dict(color='red', dash='dash', width=2),
                          marker=dict(size=8, symbol='diamond')))

fig.update_layout(
    title='🚧 Impact fermeture Champs-Élysées — Débit avant/après',
    barmode='group', template='plotly_dark', height=500,
    xaxis_title='', yaxis_title='Débit (véh/h)',
    xaxis_tickangle=-30,
)
fig.show()

In [ ]:
# ============================================================
# Carte : impact géographique de la fermeture
# ============================================================
sim1_geo = sim1.merge(axis_stats[['id_arc', 'latitude', 'longitude']], on='id_arc', how='left')

fig = px.scatter_mapbox(
    sim1_geo, lat='latitude', lon='longitude',
    size=sim1_geo['debit_apres'].clip(lower=100),
    color='surcharge',
    hover_name='nom',
    hover_data={'debit_avant': True, 'debit_apres': True, 'surcharge': ':.1f', 'status': True},
    color_continuous_scale='RdYlGn_r', range_color=[-5, 30],
    size_max=25, zoom=11.5,
    mapbox_style='carto-darkmatter',
    title='🗺️ Impact géographique — Fermeture Champs-Élysées',
    template='plotly_dark',
)
fig.update_layout(height=600)
fig.show()

In [ ]:
# ============================================================
# SCÉNARIO 2 : Fermeture du Boulevard Périphérique Porte de Vincennes
# ============================================================
sim2 = simulate_closure(axis_stats, ADJACENCY, 'AX001')
sim2[sim2['delta'] != 0][['nom', 'debit_avant', 'debit_apres', 'delta', 'taux_avant', 'taux_apres', 'surcharge', 'status']]

In [ ]:
# ============================================================
# SCÉNARIO 3 : Fermeture multiple (Rivoli + Sébastopol)
# ============================================================
def simulate_multi_closure(axis_stats, adjacency, closed_axes):
    stats = axis_stats.copy()
    total_traffic = 0
    all_neighbors = set()

    for ax in closed_axes:
        row = stats[stats['id_arc'] == ax].iloc[0]
        total_traffic += row['debit_moyen']
        for n in adjacency.get(ax, []):
            if n not in closed_axes:
                all_neighbors.add(n)

    print(f'\n🚧 FERMETURE MULTIPLE : {len(closed_axes)} axes')
    print(f'   Trafic total à redistribuer : {total_traffic:.0f} véh/h')

    results = []
    for _, row in stats.iterrows():
        r = {'id_arc': row['id_arc'], 'nom': row['nom_compteur'],
             'debit_avant': round(row['debit_moyen']), 'capacite': row['capacite'],
             'debit_apres': round(row['debit_moyen']), 'delta': 0}
        if row['id_arc'] in closed_axes:
            r['debit_apres'] = 0
            r['delta'] = -r['debit_avant']
        results.append(r)

    res_df = pd.DataFrame(results)
    n_mask = res_df['id_arc'].isin(all_neighbors)
    if n_mask.sum() > 0:
        residual = (res_df.loc[n_mask, 'capacite'] - res_df.loc[n_mask, 'debit_avant']).clip(lower=1)
        for idx in res_df[n_mask].index:
            ratio = max((res_df.loc[idx, 'capacite'] - res_df.loc[idx, 'debit_avant']) / residual.sum(), 0.05)
            added = total_traffic * ratio
            res_df.loc[idx, 'debit_apres'] = round(res_df.loc[idx, 'debit_avant'] + added)
            res_df.loc[idx, 'delta'] = round(added)

    res_df['taux_avant'] = (res_df['debit_avant'] / res_df['capacite'] * 100).round(1)
    res_df['taux_apres'] = (res_df['debit_apres'] / res_df['capacite'] * 100).round(1)
    res_df['surcharge'] = (res_df['taux_apres'] - res_df['taux_avant']).round(1)

    conditions = [res_df['id_arc'].isin(closed_axes), res_df['taux_apres'] > 90,
                  res_df['taux_apres'] > 70, res_df['taux_apres'] <= 70]
    res_df['status'] = np.select(conditions, ['⛔ FERMÉ', '🔴 Congestion', '🟡 Chargé', '🟢 Fluide'], default='🟢 Fluide')

    print(f'   ⚠️  Congestion : {(res_df["taux_apres"] > 90).sum()} axes')
    return res_df

sim3 = simulate_multi_closure(axis_stats, ADJACENCY, ['AX006', 'AX013'])
sim3[sim3['delta'] != 0][['nom', 'debit_avant', 'debit_apres', 'delta', 'taux_avant', 'taux_apres', 'status']]

---
<a id='7'></a>
## 7. Conclusions

### Résultats clés
- Les données de comptage permettent d'identifier clairement les **pics de trafic** (8h–9h et 17h–19h en semaine)
- Le **Gradient Boosting** offre les meilleures performances de prédiction du débit horaire
- La simulation de fermeture montre que les **axes du Périphérique** génèrent les surcharges les plus critiques sur les axes adjacents
- La fermeture des **Champs-Élysées** redistribue le trafic principalement sur Haussmann et Rivoli

### Limites
- Modèle de redistribution simplifié (proportionnel à la capacité résiduelle)
- Pas de prise en compte de la topologie réelle du réseau routier
- Données simulées — le pipeline est prêt à recevoir les données réelles de l'API

### Perspectives
- **Modèle de graphe** (NetworkX) pour une simulation basée sur la topologie réelle
- **Données temps-réel** via l'API Open Data Paris
- **Streamlit** pour un dashboard interactif déployable